In [1]:
# Initially at notebook directory, so step into the root directory (Run this once once)
%pwd  # in notebook directory
%cd ".." 

/scratch/htc/khiroshi/calt-dev/calt-codebase


## Load dataset

In [2]:
from sage.all import GF, PolynomialRing

import json
with open("dataset/partial_sum/GF7_n=3/test_data.jsonl", "r") as f:
    dataset = [json.loads(line) for line in f]
    
ring = PolynomialRing(GF(7), 3, "x", order="degrevlex")

Fs, Gs = [], []
for data in dataset: 
    F = list(map(lambda x: ring(x), data['problem']))  # ring() maps a polynomial in text to a polynomial object.
    G = list(map(lambda x: ring(x), data['solution']))
    Fs.append(F)
    Gs.append(G)

## Load Model

In [15]:
from pathlib import Path
from transformers import AutoModelForSeq2SeqLM
from src.utils.chekcpoint import get_checkpoint_id

use_checkpoint = True  # if True, load model from intermediate checkpoint of training
model_path = Path('results/partial_sum/GF7_n=3')

if use_checkpoint:
    checkpoint_id = get_checkpoint_id(model_path)
    model_path = model_path / f'checkpoint-{checkpoint_id}'

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_path,
)

## Generation

**Prepare dataloader**

In [ ]:
from omegaconf import OmegaConf
from calt import load_data
from torch.utils.data import DataLoader

# load training config
model_path = Path('results/partial_sum/GF7_n=3')
cfg = OmegaConf.load(model_path / 'train_example.yaml')  

dataset, tokenizer, data_collator = load_data(
        train_dataset_path=cfg.data.train_dataset_path,
        test_dataset_path=cfg.data.test_dataset_path,
        field=cfg.data.field,
        num_variables=cfg.data.num_variables,
        max_degree=cfg.data.max_degree,
        max_coeff=cfg.data.max_coeff,
        max_length=cfg.model.max_sequence_length,
    )

test_set = dataset['test']
test_loader = DataLoader(test_set, batch_size=cfg.train.test_batch_size, shuffle=False, collate_fn=data_collator)

Loaded 100000 samples from dataset/partial_sum/GF7_n=3/train_raw.txt
Loaded 1000 samples from dataset/partial_sum/GF7_n=3/test_raw.txt


**forwarding**

In [20]:
batch = next(iter(test_loader))  # get the first batch
outputs = model(**batch)

**generation**

In [33]:
outputs = model.generate(**batch)
decoded_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=True)
print(decoded_texts)  # internal representation of polynomials

from calt import PolynomialToInternalProcessor
p2i = PolynomialToInternalProcessor(num_variables=cfg.data.num_variables, max_degree=cfg.data.max_degree, max_coeff=cfg.data.max_coeff)
decoded_raw_texts = [p2i.decode(text) for text in decoded_texts]
print(decoded_raw_texts)  # raw text of polynomials

['C1 E0 E1 E0 C-2 E0 E0 E1 [SEP] C-3 E3 E2 E1 C-1 E0 E0 E2 C3 E0 E1 E0 C-2 E0 E0 E1 [SEP] C-3 E3 E2 E1 C1 E0 E0 E4 C-1 E0 E0 E2 C3 E0 E1 E0 C-2 E0 E0 E1', 'C3 E1 E0 E0 C-2 E0 E1 E0 [SEP] C2 E1 E1 E0 C3 E0 E0 E2 C1 E1 E0 E0 C-2 E0 E1 E0 C-3 E0 E0 E0 [SEP] C2 E1 E1 E0 C-2 E0 E1 E1 C3 E0 E0 E2 C1 E1 E0 E0 C2 E0 E1 E0 C2 E0 E0 E1 C-3 E0 E0 E0', 'C3 E0 E1 E0 [SEP] C-1 E1 E3 E4 C-3 E1 E0 E1 C-2 E0 E0 E2 C3 E0 E1 E0', 'C-2 E3 E1 E1 C2 E0 E0 E4 C1 E1 E1 E0 [SEP] C-2 E3 E1 E1 C2 E0 E0 E4 C1 E1 E1 E0 C-3 E1 E0 E0 [SEP] C2 E2 E2 E3 C2 E0 E0 E4 C1 E2 E1 E0 C1 E1 E1 E0 C-3 E1 E0 E0 [SEP] C2 E2 E2 E3 C2 E0 E0 E4 C1 E2 E1 E0 C1 E1 E1 E0 C2 E1 E0 E0 [SEP] C2 E2 E2 E3 C2 E0 E0 E4 C1 E2 E1 E0 C1 E1 E1 E0', 'C-3 E1 E3 E0 C-3 E1 E2 E1 C-3 E0 E1 E1 C-2 E0 E0 E1 C-2 E0 E0 E0 [SEP] C-3 E1 E3 E0 C-3 E1 E2 E1 C-2 E1 E0 E3 C-3 E0 E1 E1 C3 E0 E1 E0 C-3 E0 E0 E1 C-2 E0 E0 E0 [SEP] C-3 E4 E1 E1 C-3 E3 E1 E2 C1 E2 E0 E3 C-3 E1 E3 E0 C-3 E1 E2 E1 C-2 E1 E0 E3 C-3 E0 E1 E1 C3 E1 E0 E0 C3 E0 E1 E0 C-3 E0 E0 E1 [SEP] C

In [35]:
ring = PolynomialRing(GF(7), 3, "x", order="degrevlex")

F_text = decoded_raw_texts[0].split('|')
F = [ring(text) for text in F_text]

print(F)

[x1 - 2*x2, -3*x0^3*x1^2*x2 - x2^2 + 3*x1 - 2*x2, -3*x0^3*x1^2*x2 + x2^4 - x2^2 + 3*x1 - 2*x2]
